In [1]:
import 

def pickRadius(type):
    if (type == 1):
        
    if (type == 2):
        
    if (type == 3):
        

class disc(): 
    def __init__(self):
        

IndentationError: expected an indented block (2938115597.py, line 3)